### Pre-installs

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install seaborn
# %pip install sklearn
# %pip install -U scikit-learn
# %pip install spotipy

### Inputs

In [2]:
input_song = input("Enter your desired song name here!")
input_playlist = input("Enter your desired playlist name here!s")
recc_num = int(input("Enter how many songs you want to be recommended!"))
client_id = input("Enter your Spotify API Client Key here!")
client_secret = input("Enter your Spotify API Secret Key here!")

### CSV & Spotify Information

In [3]:
import pandas as pd
import ast

# Importing the datasets
dataset = pd.read_csv("data/tracks_with_genres.csv")
df = pd.read_csv("data/tracks.csv")

# Creating the dataframe
df_generated = pd.DataFrame(dataset)
df_generated['genres'] = df_generated['genres'].apply(ast.literal_eval)

In [4]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
# api key
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://localhost/",
                                               scope="playlist-modify-private",
                                               show_dialog=True,
                                               cache_path="token.txt"
                                               ))

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# # Assuming df_generated is your DataFrame
# # Adjust this part based on your actual DataFrame structure
# # For example, you might need to select specific columns or rows
# # or convert categorical variables to numerical values for visualization

# df_generated.columns
# df_graph = df_generated[['popularity', 'duration_ms', 'explicit',
#        'danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'time_signature']]

# sns.set(style="white")  # Set the style of the plot

# # Create a heatmap using seaborn with annotated values for the entire correlation matrix
# plt.figure(figsize=(12, 10))  # Adjust the figure size as needed
# heatmap = sns.heatmap(df_graph.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# # Customize the appearance of the heatmap
# heatmap.set_title('Heatmap of df_generated', fontsize=16)

# plt.show()

### Processing the Input Song

In [6]:
# Using Spotify API to search for a song's information based on input and adding the necessary information in a DataFrame.
def search_track(track_name):
    # Search for the track
    results = sp.search(q=track_name, type='track')

    # Check if the track exists
    if results['tracks']['total'] > 0:
        # Get the first track from the results
        track = results['tracks']['items'][0]
        explicit = int(track["explicit"] == True)
        artists = []
        id_artists = []
        for i in range (len(track["artists"])):
            artists.append(track["artists"][i]["name"])
            id_artists.append(track["artists"][i]["id"])

        print(f'Found track: {track["name"]} by {track["artists"][0]["name"]} from the album {track["album"]["name"]}.')
        track_dict = {"id": track["id"], "name": track["name"], "popularity": track["popularity"], 
                      "duration_ms": track["duration_ms"], "explicit": explicit, "artists": str(artists),
                      "id_artists": str(id_artists), "release_date": track["album"]["release_date"]}
        return track_dict
    else:
        print('Track not found')
        return None

# Creating a Single Row DataFrame for the input song.
track_result = search_track(input_song)
td = pd.DataFrame(track_result, index=[0])
td

Found track: Backlight by Ado from the album UTA'S SONGS ONE PIECE FILM RED.


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,['Ado'],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10


In [7]:
# Obtaining Feature Data from song based on its song_id from previous function and adding them to a DataFrame.
def get_audio_features(track_result):
    song_id = track_result["id"]
    results = sp.audio_features(song_id)

    if results:
        return results[0]
    else:
        print(f'No audio features found for song ID: {song_id}')
        return None

audio_features = get_audio_features(track_result)
af = pd.DataFrame(audio_features, index=[0])
# Taking out the irrevelant features.
af_formatted = af.drop(["type", "id", "uri", "track_href", "analysis_url", "duration_ms"], axis=1)
# Merging both DataFrames to sync up with the dataset's layout.
td = pd.concat([td, af_formatted], axis=1)
td['year'] = td['release_date'].str.extract(r'(\d{4})').astype(int)

In [8]:
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,['Ado'],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,-3.875,0,0.108,0.000387,0.000008,0.374,0.713,145.023,4,2022


In [9]:
import ast

td['artists'] = td['artists'].apply(ast.literal_eval)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,-3.875,0,0.108,0.000387,0.000008,0.374,0.713,145.023,4,2022


In [10]:
td['artists_count'] = len(td['artists'][0])
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,0,0.108,0.000387,0.000008,0.374,0.713,145.023,4,2022,1


In [11]:
expanded_artists = td['artists'].apply(lambda x: pd.Series(x))

# Rename the columns
expanded_artists.columns = [f"artist_{i+1}" for i in range(len(expanded_artists.columns))]

# Concatenate the expanded columns with the original DataFrame
td = pd.concat([td, expanded_artists], axis=1)
td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,0.108,0.000387,0.000008,0.374,0.713,145.023,4,2022,1,Ado


In [12]:
def get_artist_genres(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        genres = results['artists']['items'][0]['genres']
        return genres if genres else []
    except IndexError:
        return []
    
for i in range(td['artists_count'][0]):
    td['genres' + str(i+1)] = td['artist_' + str(i+1)].apply(get_artist_genres)
    td['genres' + str(i+1)] = td['genres' + str(i+1)].fillna('[]')
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres1
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,0.000387,0.000008,0.374,0.713,145.023,4,2022,1,Ado,"[j-pop, japanese teen pop]"


In [13]:
for i in range(td['artists_count'][0]):
    td['genres'] = td['genres' + str(i+1)] + td['genres' + str(i+1)]
    td.drop(['genres' + str(i+1)], axis=1, inplace=True)
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,0.000387,0.000008,0.374,0.713,145.023,4,2022,1,Ado,"[j-pop, japanese teen pop, j-pop, japanese tee..."


In [14]:
def get_unique(row):
    return list(set(row))

# Apply the function to each row of the DataFrame
td['genres'] = td['genres'].apply(get_unique)

In [15]:
td['song_name_artist'] = td['name'] + str(td['artists'])

### Adding Input to Dataset and further processing

In [16]:
# Adding the Input Song to the Dataset DataFrame. Added to the very front.
new_df = pd.concat([td, df_generated], ignore_index=True)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,tempo,time_signature,year,artists_count,artist_1,genres,song_name_artist,artist_2,artist_3,artist_4
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,145.023,4,2022,1,Ado,"[japanese teen pop, j-pop]","Backlight0 [Ado]\nName: artists, dtype: object",NaN,NaN,NaN
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,113.564,3,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,120.689,4,2020,1,The Toys,[],A Lover's Concerto['The Toys'],NaN,NaN,NaN
3,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,120.689,4,2020,1,The Toys,[thai pop],A Lover's Concerto['The Toys'],NaN,NaN,NaN
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,104.536,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN


In [17]:
new_df['song_id_name'] = new_df['id'] + new_df['name']
print(new_df.shape)
new_df.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(new_df.shape)

(228030, 29)
(186012, 29)


In [18]:
def count_items_in_list(lst):
    return len(lst)

# Apply the function to the DataFrame column
new_df['genres_count'] = new_df['genres'].apply(count_items_in_list)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,year,artists_count,artist_1,genres,song_name_artist,artist_2,artist_3,artist_4,song_id_name,genres_count
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,237666,0,[Ado],['6mEQK9m2krja6X1cfsAjfl'],2022-08-10,0.564,0.896,...,2022,1,Ado,"[japanese teen pop, j-pop]","Backlight0 [Ado]\nName: artists, dtype: object",NaN,NaN,NaN,3WY0iazRPHOlIq6CdbKLZ6Backlight,2
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,2020,1,The Toys,[],A Lover's Concerto['The Toys'],NaN,NaN,NaN,6Pkt6qVikqPBt9bEQy8iTzA Lover's Concerto,0
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9


In [19]:
print(new_df.shape)
new_df = new_df[(new_df['genres_count'] != 0)]
print(new_df.shape)

(186012, 30)
(166252, 30)


### Supersetting genres

In [20]:
# Determining the frequency of the genre. Giving the input song's genre priority to go first.
genre_extract = new_df['genres'].reset_index()
genre_wo_index = genre_extract['genres']
tally = genre_wo_index.count()
genre_dict = {}

for i in range (tally):
    input = genre_wo_index.loc[i]
    if len(input) > 0:
        for n in input:
            if n in genre_dict.keys():
                genre_dict[n] += 1
            else:
                genre_dict[n] = 1

print(genre_dict)

{'japanese teen pop': 186, 'j-pop': 4599, 'adult standards': 1216, 'bubblegum pop': 416, 'merseybeat': 529, 'rock-and-roll': 650, 'british invasion': 495, 'classic uk pop': 704, 'brill building pop': 763, 'rockabilly': 550, 'folk rock': 933, 'easy listening': 126, 'lounge': 283, 'british blues': 265, 'canadian blues': 54, 'singer-songwriter': 312, 'lilith': 307, 'canadian singer-songwriter': 107, 'folk': 393, 'rock': 3193, 'mellow gold': 1141, 'jazz': 304, 'swing': 56, 'big band': 94, 'jazz piano': 73, 'stride': 75, 'vocal jazz': 355, 'cool jazz': 274, 'harlem renaissance': 23, 'jazz saxophone': 71, 'contemporary jazz': 73, 'jazz quartet': 20, 'jazz fusion': 297, 'swedish jazz': 134, 'free jazz': 58, 'avant-garde jazz': 49, 'contemporary post-bop': 57, 'italian jazz': 15, 'soundtrack': 456, 'italian soundtrack': 28, 'vintage italian soundtrack': 34, 'classic soundtrack': 89, 'jazz trumpet': 176, 'hard bop': 204, 'roots rock': 250, 'psychedelic rock': 592, 'classic rock': 1358, 'acid ro

In [21]:
import matplotlib.pyplot as plt

# # Plot the frequency distribution
# plt.bar(genre_dict.keys(), genre_dict.values())
# plt.xlabel('Genres')
# plt.ylabel('Frequency')
# plt.title('Frequency of Various Genres')
# plt.show()

In [22]:
genres_df = pd.Series(genre_dict)
genres_df.head()

japanese teen pop     186
j-pop                4599
adult standards      1216
bubblegum pop         416
merseybeat            529
dtype: int64

In [23]:
# Summary Statistics of genres.
genres_df.T.describe()

count    4229.000000
mean      130.132655
std       393.312923
min         1.000000
25%         4.000000
50%        19.000000
75%        85.000000
max      6880.000000
dtype: float64

In [24]:
# genres_df.to_csv("data/genre_frequency.csv")

In [25]:
# Creating the genre supersets, with priority to the input song's genre.
# Since the input song is the first one to be run, its genre will be able to start the classification of other sub-genres under it.

# This can be further improved. Allowing for later genres to take over earlier genres as the superset if it more general.
# E.g.: Detroit Hip Hop is found before general hip hop. Currently they will be seperated into 2 seperate genres because the first word 'detroit' is
# not 'hip hop' and is thus overlooked.

def generate_superset_mapping(genre_dict):
    superset_mapping = {}

    for genre, frequency in genre_dict.items():
        subset_added = False
        for superset in list(superset_mapping.keys()):
            if genre in superset:
                superset_mapping[genre] = [genre]
                superset_mapping[genre].extend(superset_mapping[superset])
                del superset_mapping[superset]
                subset_added = True
                break
            elif superset in genre:
                if superset in superset_mapping:
                    superset_mapping[superset].append(genre)
                    subset_added = True
                    break
        if not subset_added:
            superset_mapping[genre] = [genre]

    return superset_mapping

# Generate superset mapping
superset_mapping = generate_superset_mapping(genre_dict)

# Output the superset mapping
for superset, subsets in superset_mapping.items():
    print(f"{superset}: {subsets}")


j-pop: ['j-pop', 'j-pop girl group', 'classic j-pop', 'j-pop boy group', 'j-poprock']
adult standards: ['adult standards', 'deep adult standards']
bubblegum pop: ['bubblegum pop']
merseybeat: ['merseybeat']
british invasion: ['british invasion']
brill building pop: ['brill building pop']
rockabilly: ['rockabilly', 'uk rockabilly', 'finnish rockabilly', 'german rockabilly', 'neo-rockabilly', 'swedish rockabilly', 'spanish rockabilly', 'rockabilly en espanol', 'japanese rockabilly', 'canadian rockabilly']
easy listening: ['easy listening']
lounge: ['lounge', 'lounge house', 'chill lounge', 'italian lounge', 'sunset lounge']
canadian blues: ['canadian blues']
singer-songwriter: ['singer-songwriter', 'canadian singer-songwriter', 'nashville singer-songwriter', 'taiwan singer-songwriter', 'singaporean singer-songwriter', 'indian singer-songwriter', 'scottish singer-songwriter', 'japanese singer-songwriter', 'swedish singer-songwriter', 'british singer-songwriter', 'turkish singer-songwriter

In [26]:
# Now with the superset mapped out, apply it onto the all the songs to determine their supersetted genre(s).
# This can be improved. Instead of only returning the first superset genre, it should return multiple in an array.

import pandas as pd

genres_df = pd.DataFrame(genres_df, columns=['genres'])

def group_into_supersets(new_df, superset_mapping):
    def get_superset(genres):
        for superset, subsets in superset_mapping.items():
            if any(subset in genre for genre in genres for subset in subsets):
                return superset
        return 'other'  # If no superset is found

    new_df['supersetted_genres'] = new_df['genres'].apply(get_superset)
    return new_df

# Group into supersets and create new column
new_df = group_into_supersets(new_df, superset_mapping)

In [27]:
new_df['supersetted_genres'].value_counts().count()

839

### Encoding

In [28]:
# One-hot Encoding of all the superset genres of each song in the dataframe.

genres_dummies = pd.get_dummies(new_df['supersetted_genres'].apply(pd.Series).stack()).groupby(level=0).sum()

# Concatenate the binary columns with the original DataFrame
df_2 = pd.concat([new_df, genres_dummies], axis=1)

In [29]:
df_2.iloc[:,0:20].describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,166252.000000,1.662520e+05,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000
mean,40.123156,2.328365e+05,0.107283,0.606616,0.653452,5.330089,-7.320521,0.609117,0.091714,0.299998,0.063974,0.207368,0.535138,121.776008,3.929324
std,17.117110,1.013132e+05,0.309474,0.157524,0.216638,3.565049,3.642161,0.487950,0.112539,0.292923,0.206059,0.183005,0.249318,29.153083,0.361895
min,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,1.911600e+05,0.000000,0.504000,0.511000,2.000000,-8.686000,0.000000,0.034200,0.039200,0.000000,0.096900,0.335000,98.006000,4.000000
50%,41.000000,2.237130e+05,0.000000,0.617000,0.681000,5.000000,-6.603000,1.000000,0.047800,0.199000,0.000002,0.131000,0.536000,121.927500,4.000000
75%,52.000000,2.624270e+05,0.000000,0.721000,0.829000,9.000000,-5.028000,1.000000,0.092800,0.514000,0.000489,0.265000,0.741000,139.952000,4.000000
max,100.000000,4.995083e+06,1.000000,0.991000,1.000000,11.000000,2.854000,1.000000,0.966000,0.996000,1.000000,1.000000,1.000000,220.099000,5.000000


In [30]:
df_2.groupby('key').count()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,yodeling,yoga,zen,zespol dzieciecy,zhongguo feng,zikir,zillertal,zolo,zouglou,zouk
key,,,,,,,,,,,,,,,,,,,,,
0,19586,19586,19586,19586,19586,0,19586,19586,19586,19586,...,19586,19586,19586,19586,19586,19586,19586,19586,19586,19586
1,14502,14502,14502,14502,14502,0,14502,14502,14502,14502,...,14502,14502,14502,14502,14502,14502,14502,14502,14502,14502
2,17337,17337,17337,17337,17337,0,17337,17337,17337,17337,...,17337,17337,17337,17337,17337,17337,17337,17337,17337,17337
3,5024,5024,5024,5024,5024,0,5024,5024,5024,5024,...,5024,5024,5024,5024,5024,5024,5024,5024,5024,5024
4,13626,13626,13626,13626,13626,0,13626,13626,13626,13626,...,13626,13626,13626,13626,13626,13626,13626,13626,13626,13626
5,13199,13199,13199,13199,13199,0,13199,13199,13199,13199,...,13199,13199,13199,13199,13199,13199,13199,13199,13199,13199
6,11252,11252,11252,11252,11252,0,11252,11252,11252,11252,...,11252,11252,11252,11252,11252,11252,11252,11252,11252,11252
7,19669,19669,19669,19669,19669,0,19669,19669,19669,19669,...,19669,19669,19669,19669,19669,19669,19669,19669,19669,19669
8,10079,10079,10079,10079,10079,0,10079,10079,10079,10079,...,10079,10079,10079,10079,10079,10079,10079,10079,10079,10079


In [31]:
df_2['key'].max()

11

In [32]:
key_encoded = pd.get_dummies(df['key'], columns=[i for i in range(12)], prefix='key').astype(int)

In [33]:
key_encoded

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
586667,1,0,0,0,0,0,0,0,0,0,0,0
586668,1,0,0,0,0,0,0,0,0,0,0,0
586669,0,0,0,0,0,0,0,1,0,0,0,0
586670,0,0,0,0,0,0,0,0,0,0,1,0


In [34]:
df_2 = pd.concat([df_2, key_encoded], axis=1)

In [35]:
print(df_2.shape)
df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(df_2.shape)

(586672, 882)
(166253, 882)


In [36]:
# Removing unnecessary features.

df_2 = df_2.drop(['id_artists', 'release_date', 'popularity', 'key', 'danceability', 'liveness', 'artists_count', 'song_name_artist', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'genres', 'supersetted_genres', 'explicit', 'time_signature'], axis=1)
df_2.head()

,id,name,duration_ms,artists,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,237666.0,[Ado],0.896,-3.875,0.0,0.1080,0.000387,0.000008,...,0,0,0,0,0,0,0,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187.0,NaN,0.265,-11.101,1.0,0.0322,0.394000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987.0,NaN,0.365,-10.226,0.0,0.0289,0.255000,0.000005,...,0,1,0,0,0,0,0,0,0,0
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093.0,NaN,0.352,-14.165,1.0,0.0300,0.406000,0.000000,...,0,0,0,1,0,0,0,0,0,0
6,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333.0,NaN,0.201,-17.796,1.0,0.0623,0.887000,0.000000,...,0,0,1,0,0,0,0,0,0,0


In [37]:
# Filling in any null values. (In case)

df_2 = df_2.fillna(0)
df_2.head()

,id,name,duration_ms,artists,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,3WY0iazRPHOlIq6CdbKLZ6,Backlight,237666.0,[Ado],0.896,-3.875,0.0,0.1080,0.000387,0.000008,...,0,0,0,0,0,0,0,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187.0,0,0.265,-11.101,1.0,0.0322,0.394000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987.0,0,0.365,-10.226,0.0,0.0289,0.255000,0.000005,...,0,1,0,0,0,0,0,0,0,0
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093.0,0,0.352,-14.165,1.0,0.0300,0.406000,0.000000,...,0,0,0,1,0,0,0,0,0,0
6,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333.0,0,0.201,-17.796,1.0,0.0623,0.887000,0.000000,...,0,0,1,0,0,0,0,0,0,0


In [38]:
df_2.shape

(166253, 866)

### Cosine Similarity & Output Extraction

In [39]:
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Feature selection, removing non-useful rows.
def processing(df_2):
    # Normalize numerical features
    numerical_features = ['loudness', 'tempo']
    scaler = MinMaxScaler()
    df_2[numerical_features] = scaler.fit_transform(df_2[numerical_features])
    
    # Standardize Year
    df_2['standardized_year'] = scaler.fit_transform(df_2[['year']])
    df_2 = df_2.drop(['year'], axis=1)

    # Create a feature matrix
    feature_matrix = df_2.drop(['id', 'name', 'artists', 'song_id_name'], axis=1)

    # Apply normalization to the entire feature matrix
    feature_matrix = scaler.fit_transform(feature_matrix)

    # This is the input song. We are isolating it from the dataframe first.
    input_song = feature_matrix[0]
    
    arr = []

    # Returns comparison value of first song in dataset to all others in an array.
    # DataFrame can be customized before Feature Selection to change which song the subject of comparison should be.
    for i in range(1, len(feature_matrix)):
        # Take the i-th song from the feature matrix
        compare_song = feature_matrix[i]
        # Perform cosine similarity.
        cosine_sim = cosine_similarity([input_song], [compare_song])
        cs_value = cosine_sim[0][0]  # Take the single similarity value between the input song and the i-th song.
        arr.append(cs_value)

    return arr

arr = processing(df_2)
arr

[0.6457705954585728,
 0.5578039955398357,
 0.4842098430085705,
 0.41353064626077896,
 0.4156460663364925,
 0.7368402213176113,
 0.5679942864428176,
 0.4689459019159957,
 0.5398037560924008,
 0.5568830632130557,
 0.6486525239446809,
 0.47094263349166615,
 0.5606689288080636,
 0.4481336456062953,
 0.44165949470588217,
 0.41625497080107077,
 0.4766528925432323,
 0.46522476546218594,
 0.6403297678953328,
 0.4548615818058057,
 0.6470003377715117,
 0.5739245005192206,
 0.5682465206379851,
 0.44492668315943285,
 0.520334179857058,
 0.500980998592117,
 0.3972218257112057,
 0.40127208166969486,
 0.6742280344186803,
 0.4104234268723588,
 0.4813778744629873,
 0.5691105273932974,
 0.5728420086728578,
 0.6371119387903814,
 0.5358574653787227,
 0.5946143575921506,
 0.6954017679348533,
 0.5235982530675289,
 0.5263408807883547,
 0.5809884258637081,
 0.7666396964646482,
 0.6688585201063115,
 0.5656756786587626,
 0.5744043873654776,
 0.5215301670569742,
 0.6494818521606847,
 0.5398508542385057,
 0.59323

In [40]:
max(arr[1:])

0.9992485662061363

In [41]:
def compile_suggestions(arr):
    suggestions = []
    
    # Compiling all song ids that are above the pre-determined threshold for 'like the first song'.
    for i in range(len(arr)):
        if arr[i] > 0.9:
            suggestions.append(i)
    
    print(suggestions)
    return suggestions
suggestions = compile_suggestions(arr)

[1087, 3567, 5774, 9136, 10946, 14227, 15790, 15995, 23674, 33791, 36991, 37434, 40079, 58020, 58349, 59289, 59337, 59530, 60420, 60883, 61479, 61564, 62009, 62360, 62714, 73417, 76809, 77326, 84650, 97644, 106691, 106907, 106952, 106978, 108082, 108099, 108734, 108747, 108816, 109018, 121749, 131705, 141258, 141324, 141485, 151626, 151636, 151649, 151680, 151762, 151795, 151804, 151813, 151824, 151852, 151861, 151866, 151899, 151900, 151901, 151939, 151945, 151988, 152018, 152031, 152125, 152156, 152160, 152164, 152179, 152193, 152227, 152232, 152275, 152354, 152356, 152383, 152386, 152429, 152437, 152440, 152450, 152464, 152474, 152483, 152511, 152538, 152548, 152559, 152626, 152633, 152701, 152741, 152756, 152781, 152784, 152794, 152802, 152871, 152882, 152906, 152982, 153006, 153010, 153054, 153098, 153100, 153114, 153151, 153169, 153170, 153217, 153243, 153244, 153310, 153362, 153395, 153462, 153463, 153473, 153482, 153486, 153496, 153572, 153597, 153630, 153639, 153652, 153662, 1

In [48]:
len(suggestions)

466

In [43]:
# Extracting out the end result information.
df_main = df_2[df_2.index.isin(suggestions)]

df_main.head()

,id,name,duration_ms,artists,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year
1087,4J4LjEe33yN6gLmPib0nqz,Time for Heroes,160280.0,0,0.925,0.888456,1.0,0.0554,0.010600,0.000211,...,0,0,0,0,1,0,0,0,0,0.989120
3567,251PNRmJU9KcUnFQAB5t6I,Kal Ho Naa Ho,321240.0,0,0.529,0.820918,1.0,0.0352,0.334000,0.000144,...,0,0,0,0,0,0,0,0,0,0.990603
5774,4mj2UMyJTBTaO7pffAK29j,Holy Mountains,328867.0,0,0.897,0.916378,1.0,0.0477,0.001000,0.004580,...,0,0,0,1,0,0,0,0,0,0.991592
15995,3Xvyu4sDpDyznUo5zjKand,Bloom - Lane 8 Remix,324437.0,0,0.587,0.819025,1.0,0.0460,0.000141,0.920000,...,1,0,0,0,0,0,0,0,0,0.996538
23674,5O2Ft5ENCfAh6JCgykJs76,Round Round,236427.0,0,0.845,0.894104,0.0,0.0338,0.002870,0.000006,...,0,0,1,0,0,0,0,0,0,0.990109


In [44]:
len(df_main) # There is one less for the last 3 features as the input song did not go through the extracted year's steps.
# df_final['standardized_year'].describe()

286

In [45]:
df_main = df_main.iloc[1:]

df_main = df_main.drop('artists', axis=1)

id_artist_mapping = df.set_index('id')['artists'].to_dict()

# Add 'artist' column to df1 using the mapping
df_main['artists'] = df_main['id'].map(id_artist_mapping)
df_main.head()

,id,name,duration_ms,energy,loudness,mode,speechiness,acousticness,instrumentalness,valence,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year,artists
3567,251PNRmJU9KcUnFQAB5t6I,Kal Ho Naa Ho,321240.0,0.529,0.820918,1.0,0.0352,0.334000,0.000144,0.295,...,0,0,0,0,0,0,0,0,0.990603,"['Shankar-Ehsaan-Loy', 'Sonu Nigam']"
5774,4mj2UMyJTBTaO7pffAK29j,Holy Mountains,328867.0,0.897,0.916378,1.0,0.0477,0.001000,0.004580,0.232,...,0,0,1,0,0,0,0,0,0.991592,['System Of A Down']
15995,3Xvyu4sDpDyznUo5zjKand,Bloom - Lane 8 Remix,324437.0,0.587,0.819025,1.0,0.0460,0.000141,0.920000,0.748,...,0,0,0,0,0,0,0,0,0.996538,"['ODESZA', 'Lane 8']"
23674,5O2Ft5ENCfAh6JCgykJs76,Round Round,236427.0,0.845,0.894104,0.0,0.0338,0.002870,0.000006,0.749,...,0,1,0,0,0,0,0,0,0.990109,['Sugababes']
33791,4LJSsa2Na4GuYzap10xf2V,Secret Effects - Xahno Remix,525133.0,0.913,0.852340,1.0,0.1420,0.009500,0.852000,0.256,...,0,0,0,0,0,0,1,0,0.994560,['Phaxe']


### Recommendations Output

In [46]:
# Generating recc_num of the matched songs at random.
def recommend(df_main, recc_num):
    recs = []
    if len(df_main) == 0:
        print("Sorry! There are no songs similar enough to " + input_song + "!")
    elif len(df_main) < recc_num:
        recc_num = len(df_main)
        print("Uh oh! We only found " + str(recc_num) + " recommendations!")
    print('Here are your recommendations!')
    for c in range(recc_num):
        recc = df_main.sample().to_dict()
        name = list(recc['name'].values())[0]
        artists = (list(recc['artists'].values())[0])[1:len(list(recc['artists'].values())[0])-1]
        year = str(list(recc['year'].values())[0])
        print(str(c + 1) + '. ' + name + ' by ' + artists + ' published in ' + year)
        df_main = df_main.drop(list(recc['id'])[0])
        recs.append((name, year))
    return recs
# May want to try a filter for repreventing duplicate songs in the list.
recs_for_user = recommend(df_main, recc_num) # End Product

Here are your recommendations!
1. Pode Ser by 'Pedro Mariano' published in 2002.0
2. Píntame de Azul by 'Elefante' published in 2005.0
3. Ploaia by 'El Negro' published in 2010.0
4. Un Hombre Con Suerte by 'Duelo' published in 2016.0
5. Olhos verdes by 'Amado Batista' published in 2001.0
6. Acho Melhor Ir De Uma Vez by 'Noda de Caju' published in 2002.0
7. Eslabon Por Eslabon by 'Valentín Elizalde' published in 2002.0
8. Debaixo das Asas by 'Voz da Verdade' published in 2002.0
9. Todos Locos by 'Los Caligaris' published in 2015.0
10. Hoteluri by 'B.U.G. Mafia', 'Mario V' published in 2009.0


In [47]:
user_id = sp.current_user()["id"]
song_uris = []
for song in recs_for_user:
    result = sp.search(q=f"track:{song[0]} year:{format(float(song[1]), '.0f')}", type="track")
    print(result)
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song[0]} doesn't exist in Spotify. Skipped.")

playlist = sp.user_playlist_create(user=user_id, name=input_playlist, public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3APode+Ser+year%3A2002&type=track&offset=0&limit=10', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3flF2SJ6De8yPikxEI5bpY'}, 'href': 'https://api.spotify.com/v1/artists/3flF2SJ6De8yPikxEI5bpY', 'id': '3flF2SJ6De8yPikxEI5bpY', 'name': 'Pedro Mariano', 'type': 'artist', 'uri': 'spotify:artist:3flF2SJ6De8yPikxEI5bpY'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'M

{'snapshot_id': 'MiwyODkwZDZkNjkyZjllZjBjYWI3NGFmYWNiZTA2ZTIzZjk4ZmFiOWYw'}